# First simulation

A (Monte Carlo) Simulation consists in (1) [options](#setting-up-simulation-options), defining all the necessary parameters to setup the simulation, and (2) [results](#results), containing all the outputs of a simulation. One or more simulations form a `Project`. A **pyMonteCarlo** project stored on disk has the extension `.mcsim`. It consists of a [HDF5](http://hdf5group.org) file and can be opened in the [HDFViewer](http://hdf5group.org) or using any HDF5 library.

## Setting up simulation options

The options are defined by the class `Options`. It contains all the parameters necessary to run a simulation. The parameters are grouped into four categories: 

* program
* beam
* sample
* analyses

The beam, sample and analyses are independent of Monte Carlo programs. In other words, the same sample definition can be used for different Monte Carlo programs. For a given `Options` instance, only the program needs to change to run the same simulation with different Monte Carlo programs. That being said not all beam, sample and analyses are supported by all Monte Carlo programs. Supported parameters for each Monte Carlo program are listed in the [supported options](../supported-options.rst) page.

### Program

The program is specific to a particular Monte Carlo program. Each program follows the contract specified by the base program class `Program`. One implementation is for [Casino 2](http://www.gel.usherbrooke.ca/casino) as part of the package **pymontecarlo-casino2**. The program can be imported as follow:

In [4]:
from pymontecarlo_casino2.program import Casino2Program

The parameters associated with the program will depend on each Monte Carlo program. For Casino 2, the number of trajectories and the models used for the simulation can be specified. Here is an example with the default models and 5000 trajectories:

In [5]:
program = Casino2Program(5000)

Throughout **pyMonteCarlo**, a parameter can also be set/modified using its attribute inside the class:

In [6]:
program.number_trajectories = 6000

All parameters are completely mutable and are only validated before a simulation starts.

### Beam

The second category of parameters is the beam. At the moment, two types of beam are implemented/supported: a Gaussian and a cylindrical beam. All beam implementations must define the energy and type of the incident particles as defined by the base `Beam` class. The type of incident particle is defined for future expansions, since all  currently supported Monte Carlo programs only accept `ELECTRON`. Unless otherwise stated, all beams assume that the incident particles travel downwards along the z-axis, i.e. following the vector `(0, 0, -1)`.

The Gaussian beam is the most supported by the different Monte Carlo programs. Besides the incident energy, the diameter corresponding to the full width at half maximum (FWHM) of a two dimensional Gaussian distribution must be specified. For a 15kV beam with a diameter of 10nm:

In [7]:
from pymontecarlo.options.beam.gaussian import GaussianBeam
beam = GaussianBeam(15e3, 10e-9)

Other parameters of the beam are the beam center position. The actual position of each particle will be randomly sampled by the Monte Carlo program to obtain a two dimensional Gaussian distribution centered at the specified position. By default, the beam is centered at `x = 0m` and `y = 0m`. The position can be changed using either attribute:

In [8]:
beam.x_m = 100e-9
beam.y_m = 200e-9

If you are looking to perform a line scan, you should have a look at the Gaussian beam builder class `GaussianBeamBuilder`. Builder classes are helper classes to create multiple instances by varying one or more parameters. For example, if we would like to create a Gaussian beam at two incident energy (5 and 15kV) and scan the sample from -100 to 100μm with a step size of 25μm:

In [9]:
from pymontecarlo.options.beam.gaussian import GaussianBeamBuilder
builder = GaussianBeamBuilder()
builder.add_energy_eV(5e3)
builder.add_energy_keV(15)
builder.add_diameter_m(10e-9)
builder.add_linescan_x(-100e-6, 100e-6, 25e-6)
beams = builder.build()

The variable `beams` is a `list` of 16 Gaussian beams, 2 incident electron energies and 8 positions in the linescan. Note however that each `Options` instance can only take one beam. This will result in an *exception* at validation:

    options.beam = beams # Incorrect